# Task 30.04.20 Give Me Some Credit. Применение hyperopt, scopt, optuna

In [39]:
#Выбросы (взгляд из Excel)
#NumberOfTime30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate, NumberOfTime60-89DaysPastDueNotWorse = 96 или 98
#Очень большой DebtRatio, как правило, при MonthlyIncome = NA или 0. Импутация MonthlyIncome ситуацию не исправит, 
#т.к. не приведёт к пересчету DebtRatio.
#И пр.


In [40]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import GradientBoostingRegressor
#from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
#from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
#from category_encoders import WOEEncoder, SumEncoder
import matplotlib.pyplot as plt
%matplotlib inline
from lightgbm import LGBMClassifier
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from numpy.random import RandomState
import warnings
warnings.filterwarnings("ignore")

In [41]:
os.getcwd()

'D:\\ML\\Gewissta\\My course\\GiveMeSomeCredit'

In [42]:
os.chdir('D:\\ML\\Gewissta\\My course\\GiveMeSomeCredit')

In [43]:
data = pd.read_csv('cs-training.csv', sep = ',')

In [44]:
data.tail()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
149995,149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0
149999,150000,0,0.850283,64,0,0.249908,8158.0,8,0,2,0,0.0


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            150000 non-null  int64  
 1   SeriousDlqin2yrs                      150000 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 3   age                                   150000 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 5   DebtRatio                             150000 non-null  float64
 6   MonthlyIncome                         120269 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 8   NumberOfTimes90DaysLate               150000 non-null  int64  
 9   NumberRealEstateLoansOrLines          150000 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 11  

In [46]:
data.isnull().sum()

Unnamed: 0                                  0
SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [47]:
data.duplicated().sum()

0

# Hyperopt

In [48]:
num_columns_1 = ['MonthlyIncome']
num_columns_2 = ['NumberOfDependents']
num_columns_3 = ['RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 
                'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 
                'NumberOfTime60-89DaysPastDueNotWorse']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis = 1), data['SeriousDlqin2yrs'], 
                                                   test_size = 0.3, stratify = data['SeriousDlqin2yrs'],
                                                   random_state = 42)

In [50]:
num_pipe_1 = Pipeline([('imp', SimpleImputer(strategy = 'mean')), 
                    ('scaler', StandardScaler())])

In [51]:
num_pipe_2 = Pipeline([('imp', SimpleImputer(strategy = 'most_frequent')), 
                    ('scaler', StandardScaler())])

In [52]:
num_pipe_3 = Pipeline([('scaler', StandardScaler())])

In [53]:
transformers = [('num_1', num_pipe_1, num_columns_1), ('num_2', num_pipe_2, num_columns_2), 
                ('num_3', num_pipe_3, num_columns_3)]

In [54]:
transformer = ColumnTransformer(transformers = transformers)

In [55]:
ml_pipe = Pipeline([('tf', transformer),
                    ('logreg', LogisticRegression(solver = 'lbfgs', max_iter = 200))])

In [18]:
def hyperopt_objective(params):
    ml_pipe.set_params(**params)
    return cross_val_score(ml_pipe, X_train, y_train, cv=5).mean()

In [19]:
space_hyperopt = {
    'logreg__C': hp.loguniform('logreg__C', np.log(0.01), np.log(100)),
    'logreg__solver': hp.choice('logreg__solver', ['lbfgs', 'liblinear']),
    'tf__num_1__imp__strategy': hp.choice(
        'tf__num_1__imp__strategy', ['mean', 'median', 'constant']),
    'tf__num_2__imp__strategy': hp.choice(
        'tf__num_2__impr__strategy', ['most_frequent', 'constant'])
}

In [20]:
def f(params): ########
    acc = hyperopt_objective(params) ########
    return {'loss': -acc, 'status': STATUS_OK}

best = fmin(fn=f, 
            space=space_hyperopt,
            algo=tpe.suggest, 
            max_evals=50,
            rstate=RandomState(123))

print('best:')
print(best)

100%|███████████████████████████████████████████████| 50/50 [03:03<00:00,  3.91s/trial, best loss: -0.9339714361613864]
best:
{'logreg__C': 1.587197399609853, 'logreg__solver': 1, 'tf__num_1__imp__strategy': 0, 'tf__num_2__impr__strategy': 0}


In [21]:
best_params = space_eval(space_hyperopt, best)
ml_pipe.set_params(**best_params)
ml_pipe.fit(X_train, y_train)
test_predict = ml_pipe.predict(X_test)
test_predict_proba = ml_pipe.predict_proba(X_test)
acc = accuracy_score(y_test, test_predict)
print(test_predict[:10])
print(test_predict_proba[:10])
print(acc)

[0 0 0 0 0 0 0 0 0 0]
[[0.97245442 0.02754558]
 [0.96140841 0.03859159]
 [0.95591205 0.04408795]
 [0.95030331 0.04969669]
 [0.9725738  0.0274262 ]
 [0.91988212 0.08011788]
 [0.89942574 0.10057426]
 [0.91819734 0.08180266]
 [0.94394718 0.05605282]
 [0.95939253 0.04060747]]
0.9338222222222222


# Skopt

In [22]:
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Real, Categorical, Integer

In [23]:
space_skopt = {'logreg__C': Real(np.log10(1.01), np.log10(100), 'log_uniform'),
    'logreg__solver': Categorical(['lbfgs', 'liblinear']),
    'tf__num_1__imp__strategy': Categorical(['mean', 'median', 'constant']),
    'tf__num_2__imp__strategy': Categorical(['most_frequent', 'constant'])}

In [24]:
opt = BayesSearchCV(ml_pipe,
                    space_skopt,
                    n_iter=10,
                    random_state=42,
                    cv=5)
opt.fit(X_train, y_train)
print('best_score:', opt.best_score_)

best_score: 0.9339428571428572


In [25]:
results = pd.DataFrame(opt.cv_results_)
tbl = results.pivot_table(values=['mean_test_score'],    
                          index=['param_logreg__C', 
                                 'param_logreg__solver',
                                 'param_tf__num_1__imp__strategy',
                                 'param_tf__num_2__imp__strategy'])
print(tbl)

                                                                                                    mean_test_score
param_logreg__C param_logreg__solver param_tf__num_1__imp__strategy param_tf__num_2__imp__strategy                 
0.004419        liblinear            constant                       most_frequent                          0.933295
0.053545        liblinear            constant                       most_frequent                          0.933638
0.066265        liblinear            mean                           most_frequent                          0.933705
0.121342        liblinear            median                         constant                               0.933790
0.190717        liblinear            median                         constant                               0.933886
0.390936        liblinear            mean                           most_frequent                          0.933933
0.584464        lbfgs                median                         cons

In [26]:
print(opt.best_score_) #повторно
print(opt.best_params_)
print(opt.score(X_test, y_test))

0.9339428571428572
OrderedDict([('logreg__C', 0.5844644531295251), ('logreg__solver', 'lbfgs'), ('tf__num_1__imp__strategy', 'median'), ('tf__num_2__imp__strategy', 'constant')])
0.9338222222222222


In [27]:
# Перепроверка. Расчет с другой стороны
best_params_skopt = opt.best_params_

ml_pipe.set_params(**best_params_skopt)
ml_pipe.fit(X_train, y_train)

test_predict = ml_pipe.predict(X_test)

acc = accuracy_score(y_test, test_predict)
print(acc)

0.9338222222222222


In [28]:
#Поиск с помощью декоратора @use_named_args
from skopt.utils import use_named_args

In [30]:
space_skopt2 = [
    Real(np.log10(1.01), np.log10(100), name = 'logreg__C'),
    Categorical(['lbfgs', 'liblinear'], name = 'logreg__solver'),
    Categorical(['mean', 'median', 'constant'], name = 'tf__num_1__imp__strategy'),
    Categorical(['most_frequent', 'constant'], name = 'tf__num_2__imp__strategy')
]

In [33]:
# пишем функцию, возвращающую правильность,
# усредненную по итогам перекрестной проверки
@use_named_args(space_skopt2)
def objective(**params):
    ml_pipe.set_params(**params)
    return -np.mean(cross_val_score(ml_pipe,
                                    X_train,
                                    y_train,
                                    cv=5,
                                    n_jobs=-1))

In [34]:
res = gp_minimize(objective, space_skopt2, n_calls=10, random_state=42)

In [36]:
# смотрим наилучшее значение
print('Наилучшее значение AUC: {:.4f}'.format(np.abs(res.fun)))### Why AUC?
# смотрим наилучшие гиперпараметры
pd.DataFrame({'logreg__C': pd.Series(res.x[0]),
              'logreg__solver': pd.Series(res.x[1]),
              'tf__num_1__imp__strategy': pd.Series(res.x[2]),
              'tf__num_2__imp__strategy': pd.Series(res.x[3])})

Наилучшее значение AUC: 0.9340


,logreg__C,logreg__solver,tf__num_1__imp__strategy,tf__num_2__imp__strategy
0,1.224985,lbfgs,mean,constant


# Optuna

In [59]:
import optuna

AttributeError: module 'optuna' has no attribute 'structs'